In [ ]:
import os, numpy, torch

from matplotlib import pyplot as plt
from matplotlib import colors, colormaps
#brain visualization import
from nilearn import regions, datasets, surface, plotting, image, maskers
from nilearn.plotting import plot_roi, plot_stat_map

In [ ]:
MIST_path = '/home/maellef/DataBase/fMRI_parcellations/MIST_parcellation/Parcellations/MIST_ROI.nii.gz'
voxel_mask = '/home/maellef/git/cNeuromod_encoding_2020/parcellation/STG_middle.nii.gz'
dir_path = "/home/maellef/Results/Phantom_general"

In [ ]:
def voxels_nii(voxel_data, voxel_mask, t_r=1.49):
#from voxels to nii
    voxel_masker = maskers.NiftiMasker(mask_img=voxel_mask, standardize=False, 
                                       detrend=False, t_r=t_r, smoothing_fwhm=8)
    voxel_masker.fit()
    vox_data_nii = voxel_masker.inverse_transform(voxel_data)
    return vox_data_nii

In [ ]:
def surface_fig(parcel_data, vmax, threshold=0, cmap='turbo', inflate=True, colorbar=True, no_background=True):     
    nii_data = regions.signals_to_img_labels(parcel_data, MIST_path)
    fig, ax = plotting.plot_img_on_surf(nii_data,
                              views=['lateral', 'medial'], hemispheres=['left', 'right'], inflate=inflate,
                              vmax=vmax, threshold=threshold, colorbar=colorbar, cmap=cmap, symmetric_cbar=False)
    return fig

In [ ]:
def voxel_map(voxel_data, vmax=None, cut_coords=None, tr = 1.49, bg_img=None, cmap = 'cold_hot') : 
    f = plt.Figure()
    data_nii = voxels_nii(voxel_data, voxel_mask, t_r=tr)
    if bg_img is not None : 
        plotting.plot_stat_map(data_nii, bg_img=bg_img, draw_cross=False, vmax=vmax,
                           display_mode='x', cut_coords=[-63, -57, 57, 63], figure=f,
                              black_bg=True, dim = 0, cmap=cmap)
    else :
        plotting.plot_stat_map(data_nii, draw_cross=False, vmax=vmax,
                           display_mode='x', cut_coords=[-63, -57, 57, 63], figure=f)
    return f

In [ ]:
def extend_colormap(original_colormap = 'twilight', percent_start = 0.25, percent_finish = 0.25):
    colormap = colormaps[original_colormap]
    nb_colors = colormap.N
    new_colors_range = colormap(numpy.linspace(0,1,nb_colors))

    n_start = round(nb_colors/(1-percent_start)) - nb_colors if percent_start != 0 else 0
    new_color_start = numpy.array([colormap(0)]*n_start).reshape(-1, new_colors_range.shape[1])
    n_finish = round(nb_colors/(1-percent_finish)) - nb_colors if percent_finish != 0 else 0
    new_color_finish = numpy.array([colormap(0)]*n_finish).reshape(-1, new_colors_range.shape[1])

    new_colors_range = numpy.concatenate((new_color_start,new_colors_range,new_color_finish), axis=0)
    new_colormap = colors.ListedColormap(new_colors_range)
    return new_colormap

In [ ]:
for dataset in ['friends', 'movie10']:
    subdatasets = ['s04'] if dataset == 'friends' else ['bourne', 'figures', 'life', 'wolf']
    for sub_ds in subdatasets:
        dspath = os.path.join(dir_path, dataset, sub_ds)
        for file in os.listdir(dspath):
            sub = file[:6]
            conv = 'conv4' if 'conv4' in file else 'baseline'
            scale = 'WB' if 'MIST_ROI' in file else 'STG'
            
            filepath = os.path.join(dspath, file)
            arr = numpy.load(filepath)
            print(arr.shape)

            parcel_data = numpy.mean(arr, axis=0).reshape(1, -1)
            vmax = numpy.max(numpy.abs(parcel_data))
            print(dataset, sub_ds, sub, scale, conv)
            print(f'min :', numpy.min(parcel_data), f', max : ', numpy.max(parcel_data), f', absolute max : ', vmax)

            #cmp = extend_colormap(original_colormap = 'turbo', percent_start=0.5, percent_finish=0) 
            if scale == 'WB' :
                fig = surface_fig(parcel_data, vmax=None, threshold = 0.1)
            else :
                fig = voxel_map(parcel_data)
        
            output_file = os.path.join(dir_path, 'maps', '{}_{}_{}_{}_{}'.format(dataset, sub_ds, sub, scale, conv))
            fig.savefig(output_file, dpi=100)